In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def draw_garden(n, blue, smpl):
    sn = len(smpl)

    positions_lst = [np.arange(1, n**x + 1) / n**(x-1) for x in range(1, sn+1)]
    positions = np.concatenate(positions_lst)

    d = pd.DataFrame({
        'position': positions,
        'draw': np.repeat(np.arange(1, sn+1), [n**i for i in range(1, sn+1)]),
        'fill': np.tile(np.repeat(['b', 'w'], [blue, n - blue]), sum([n**i for i in range(sn)])),
        'pos_char': positions.astype(str)
    })

    lines_lst = [
        pd.DataFrame({
            'x': np.repeat(np.arange(1, n**s + 1) / n**(s-1), n),
            'xend': np.arange(1, n**(s+1) + 1) / n**s,
            'y': s,
            'yend': s + 1,
            'draw': s
        }) for s in range(1, sn)
    ]

    lines_dt = pd.concat(lines_lst).merge(
        d[['draw', 'position', 'fill']].rename(columns={'fill': 'fill_start'}),
        left_on=['draw', 'x'], right_on=['draw', 'position']
    ).merge(
        d.assign(draw=d['draw'] - 1)[['draw', 'position', 'fill']].rename(columns={'fill': 'fill_end'}),
        left_on=['draw', 'xend'], right_on=['draw', 'position']
    ).assign(
        fill_smpl=lambda df: smpl[df['draw']],
        fill_smpl_next=lambda df: smpl[df['draw'] + 1],
        visible=lambda df: np.where(df['fill_start'] == df['fill_smpl'], np.where(df['fill_end'] == df['fill_smpl_next'], 1, 0), 0)
    )

    lines_dt = lines_dt.merge(
        lines_dt[['draw', 'visible', 'xend']].rename(columns={'visible': 'parent_visible'}).assign(draw=lambda df: df['draw'] + 1),
        left_on=['draw', 'x'], right_on=['draw', 'xend'], how='left'
    ).assign(
        visible=lambda df: np.where(df['parent_visible'].isna(), df['visible'], np.where(df['parent_visible'] == 0, 0, df['visible']))
    ).assign(
        x=lambda df: df['x'] - 0.5 / n**(df['draw'] - 1),
        xend=lambda df: df['xend'] - 0.5 / n**df['draw']
    )

    d = d.assign(
        denominator=lambda df: np.where(df['draw'] == 1, 0.5, np.where(df['draw'] == 2, 0.5 / n, 0.5 / n**2)),
        position=lambda df: df['position'] - df['denominator']
    )

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    for _, row in lines_dt.iterrows():
        ax.plot([row['x'], row['xend']], [row['y'], row['yend']], color='black', linewidth=1/3, alpha=row['visible'])

    colors = {'b': 'navy', 'w': 'white'}
    for _, row in d.iterrows():
        ax.scatter(row['position'], row['draw'], color=colors[row['fill']], s=100, edgecolor='black')

    ax.set_ylim(0.75, sn)
    ax.set_yticks([])
    ax.set_xticks([])

    plt.show()

    return d, lines_dt

# Example usage
draw_garden(4, 2, ["b", "w", "b"])

TypeError: list indices must be integers or slices, not Series

In [13]:
sn = 3
blue = 2
n = 4

positions_lst = [np.linspace(0, 4, n**x) for x in range(1, sn+1)]

In [11]:
for x in range(1, 4):
    print(x)

1
2
3
